In [6]:
from craigslist import CraigslistForSale
import pandas as pd
import webbrowser
import smtplib
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

In [7]:
def sendemail(from_addr, to_addr_list, cc_addr_list,
              subject, message,
              login, password,
              smtpserver='smtp.gmail.com:587'):
    header  = 'From: %s ' % from_addr + "\n"
    header += 'To: %s ' % ','.join(to_addr_list) + "\n"
    header += 'Cc: %s ' % ','.join(cc_addr_list) + "\n"
    header += 'Subject: %s ' % subject + "\n"
    message = header + message
 
    server = smtplib.SMTP(smtpserver)
    server.starttls()
    server.login(login,password)
    problems = server.sendmail(from_addr, to_addr_list, message)
    server.quit()

def search_craigslist_for_cars(loc,car,minyear,maxyear,minprice,maxprice,openlinks,email):
    msg = ''
    #Pull data
    cars = CraigslistForSale(site=loc, area='', category='cars-trucks', 
                         filters={'min_price': minprice, 'max_price': maxprice, 'min_year':minyear, 'max_year':maxyear,'query':car})
    #Print Title
    print( color.BOLD + 
          str(cars.filters['min_auto_year']) + ' or newer ' + cars.filters['query'] + ' between ' + 
          str(cars.filters['min_price']) + ' and ' + str(cars.filters['max_price']) + ' in ' + loc + ' area '
    + color.END)
    
    #Print search results and open websites
    for x in cars.get_results():
        #Only return results if url contains one of the words from the search critera
        if len(set(car.split(' ')) & set( x['url'].split('/')[5].split('-'))) > 1:
            print('Posted:', x['datetime'], 'Price:', x['price'], 'Link:' , x['url'])
            msg += 'Posted: ' + x['datetime'] + 'Price:' + x['price'] + ' Link:' + x['url']
            
            if openlinks == True:
                webbrowser.open(x['url'])
    print('')
    
    if email == True:
        sendemail(from_addr = 'thefifthfox0@gmail.com', 
        to_addr_list = ['qman2386@gmail.com'],
        cc_addr_list = [''], 
        subject      =  'Three quotes v2', 
        message      =  msg, 
        login        = 'thefifthfox0@gmail.com', 
        password     = 'computerfox')

In [8]:
sites = ['abilene','akroncanton','albany','albanyga','albuquerque','allentown','altoona','amarillo','ames','anchorage','annapolis','annarbor','appleton','asheville','ashtabula','athensga','athensohio','atlanta','auburn','augusta','austin','bakersfield','baltimore','batonrouge','battlecreek','beaumont','bellingham','bemidji','bend','bgky','bham','bigbend','billings','binghamton','bismarck','blacksburg','bloomington','bn','boise','boone','boston','boulder','bozeman','brainerd','brownsville','brunswick','buffalo','butte','capecod','carbondale','catskills','cedarrapids','cenla','centralmich','cfl','chambana','chambersburg','charleston','charlestonwv','charlotte','charlottesville','chattanooga','chautauqua','chicago','chico','chillicothe','cincinnati','clarksville','cleveland','clovis','cnj','collegestation','columbia','columbiamo','columbus','columbusga','cookeville','corpuschristi','corvallis','cosprings','csd','dallas','danville','dayton','daytona','decatur','delaware','delrio','denver','desmoines','detroit','dothan','dubuque','duluth','eastco','easternshore','eastidaho','eastky','eastnc','eastoregon','easttexas','eauclaire','elko','elmira','elpaso','enid','erie','eugene','evansville','fairbanks','fargo','farmington','fayar','fayetteville','fingerlakes','flagstaff','flint','florencesc','fortcollins','fortdodge','fortlauderdale','fortmyers','fortsmith','fortwayne','frederick','fredericksburg','fresno','gadsden','gainesville','galveston','glensfalls','goldcountry','grandforks','grandisland','grandrapids','greatfalls','greenbay','greensboro','greenville','gulfport','hanford','harrisburg','harrisonburg','hartford','hattiesburg','helena','hickory','hiltonhead','holland','honolulu','houma','houston','hudsonvalley','humboldt','huntington','huntsville','miami','imperial','indianapolis','inlandempire','iowacity','ithaca','jackson','jacksontn','jacksonville','janesville','jerseyshore','jonesboro','joplin','juneau','jxn','kalamazoo','kalispell','kansascity','kenai','keys','killeen','kirksville','klamath','knoxville','kokomo','kpr','ksu','lacrosse','lafayette','lakecharles','lakecity','lakeland','lancaster','lansing','laredo','lasalle','lascruces','lasvegas','lawrence','lawton','lewiston','lexington','limaohio','lincoln','littlerock','logan','longisland','losangeles','louisville','loz','lubbock','lynchburg','macon','madison','maine','mankato','mansfield','marshall','martinsburg','masoncity','mattoon','mcallen','meadville','medford','memphis','mendocino','merced','meridian','miami','milwaukee','minneapolis','missoula','mobile','modesto','mohave','monroe','monroemi','montana','monterey','montgomery','morgantown','moseslake','muncie','muskegon','myrtlebeach','nacogdoches','nashville','natchez','nd','nesd','newhaven','newjersey','newlondon','neworleans','newyork','nh','nmi','norfolk','northernwi','northmiss','northplatte','nwct','nwga','nwks','ocala','odessa','ogden','okaloosa','oklahomacity','olympic','omaha','oneonta','onslow','orangecounty','oregoncoast','orlando','ottumwa','outerbanks','owensboro','palmsprings','panamacity','parkersburg','pennstate','pensacola','peoria','philadelphia','phoenix','pittsburgh','plattsburgh','poconos','porthuron','portland','potsdam','prescott','providence','provo','pueblo','pullman','quadcities','quincy','racine','raleigh','rapidcity','reading','redding','reno','richmond','richmondin','rmn','roanoke','rochester','rockford','rockies','roseburg','roswell','sacramento','saginaw','salem','salina','saltlakecity','sanangelo','sanantonio','sandiego','sandusky','sanmarcos','santabarbara','santafe','santamaria','sarasota','savannah','scottsbluff','scranton','sd','seattle','seks','semo','sfbay','sheboygan','shoals','showlow','shreveport','sierravista','siouxcity','siouxfalls','siskiyou','skagit','slo','smd','southbend','southcoast','southjersey','spacecoast','spokane','springfield','springfieldil','statesboro','staugustine','stcloud','stgeorge','stillwater','stjoseph','stlouis','stockton','susanville','swks','swmi','swv','swva','syracuse','tallahassee','tampa','terrehaute','texarkana','texoma','thumb','tippecanoe','toledo','topeka','treasure','tricities','tucson','tulsa','tuscaloosa','tuscarawas','twinfalls','twintiers','up','utica','valdosta','ventura','vermont','victoriatx','visalia','waco','washingtondc','waterloo','watertown','wausau','wenatchee','westernmass','westky','westmd','westslope','wheeling','wichita','wichitafalls','williamsport','wilmington','winchester','winstonsalem','worcester','wv','wyoming','yakima','york','youngstown','yubasutter','yuma','zanesville']

In [10]:
already_checked = ['https://anchorage.craigslist.org/cto/d/1978-porsche-911-sc-only-5000/6680836329.html','https://austin.craigslist.org/cto/d/1976-porsche-911-convertible/6664202810.html','https://bakersfield.craigslist.org/cto/d/911-porsche-convertible/6686260433.html','https://bham.craigslist.org/cto/d/1983-porsche-911-bridgestone/6687907347.html','https://boise.craigslist.org/cto/d/rare-1970-porsche-911-body/6649790854.html','https://charleston.craigslist.org/cto/d/1986-porsche-911-conv/6649734485.html','https://cleveland.craigslist.org/cto/d/1983-porsche-911-goodyear/6687914825.html','https://columbia.craigslist.org/cto/d/1983-burgundy-porsche-911/6687196348.html','https://daytona.craigslist.org/cto/d/1986-porsche-911-carrera-conv/6649761594.html','https://eastoregon.craigslist.org/cto/d/early-porsche-911-project-wtb/6679741734.html','https://fargo.craigslist.org/cto/d/1983-porsche-911-garaged-with/6682299789.html','https://flint.craigslist.org/cto/d/1983-porsche-911-burgundy/6687285576.html','https://grandrapids.craigslist.org/cto/d/porsche-911-project-or-fixer/6685967106.html','https://greenville.craigslist.org/cto/d/1986-porsche-911-conv/6649701036.html','https://houston.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678163728.html','https://hudsonvalley.craigslist.org/cto/d/1982-porsche-911-slant-nose/6681725992.html','https://inlandempire.craigslist.org/cto/d/1983-porsche-911-driver-airbag/6687979723.html','https://lansing.craigslist.org/cto/d/911-porsche-looking-for-older/6687104481.html','https://lasvegas.craigslist.org/cto/d/1987-porsche-911-carrera/6687002558.html','https://lasvegas.craigslist.org/cto/d/1969-porsche-911-baja-race/6668904623.html','https://lasvegas.craigslist.org/cto/d/1977-porsche-911-sc-targa/6680151081.html','https://monterey.craigslist.org/cto/d/red-1977-porsche-911-sc-targa/6680149748.html','https://monterey.craigslist.org/ctd/d/1984-porsche-911-cab-guards/6677467366.html','https://nmi.craigslist.org/cto/d/69-porsche-911-irish-green/6656476893.html','https://norfolk.craigslist.org/cto/d/1972-porsche-911/6663694575.html','https://orlando.craigslist.org/cto/d/1991-porsche-911-carrera-2/6680295712.html','https://pensacola.craigslist.org/cto/d/1983-porsche-911-manual/6684020499.html','https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678187037.html','https://philadelphia.craigslist.org/cto/d/1974-porsche-911-targa/6670015872.html','https://reno.craigslist.org/cto/d/porsche-911/6683051897.html','https://sacramento.craigslist.org/cto/d/1967-porsche-911/6682711263.html']

In [25]:
#solve for if link contains a word with no space like 
search_craigslist_for_cars('philadelphia','porsche 911',1900,1998,0,50000,False,False)

1900 or newer porsche 911 between 0 and 50000 in philadelphia area 
Posted: 2018-08-23 16:41 Price: $50000 Link: https://philadelphia.craigslist.org/cto/d/porsche-911-targa-yellow-70/6679008664.html
Posted: 2018-08-22 17:41 Price: $24900 Link: https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678187037.html
Posted: 2018-08-21 14:49 Price: $49500 Link: https://philadelphia.craigslist.org/ctd/d/1992-porsche-911-c2-964-targa/6677113665.html
Posted: 2018-08-13 15:09 Price: $9000 Link: https://philadelphia.craigslist.org/cto/d/1974-porsche-911-targa/6670015872.html
Posted: 2018-08-06 12:03 Price: $38900 Link: https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-sc-67k/6663544646.html



In [ ]:
#Search all sites for data
for x in sites:
    search_craigslist_for_cars(x,'porsche 911',1900,1998,1000,30000,True,False)

In [13]:
#Testing
cars = CraigslistForSale(site='philadelphia', area='', category='cars-trucks', 
                         filters={'min_price': 0, 'max_price': 30000, 'min_year':1900, 'max_year':1998,'query':'porsche 911'})

In [24]:
#Test: https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678187037.html
already_checked = ['https://anchorage.craigslist.org/cto/d/1978-porsche-911-sc-only-5000/6680836329.html','https://austin.craigslist.org/cto/d/1976-porsche-911-convertible/6664202810.html','https://bakersfield.craigslist.org/cto/d/911-porsche-convertible/6686260433.html','https://bham.craigslist.org/cto/d/1983-porsche-911-bridgestone/6687907347.html','https://boise.craigslist.org/cto/d/rare-1970-porsche-911-body/6649790854.html','https://charleston.craigslist.org/cto/d/1986-porsche-911-conv/6649734485.html','https://cleveland.craigslist.org/cto/d/1983-porsche-911-goodyear/6687914825.html','https://columbia.craigslist.org/cto/d/1983-burgundy-porsche-911/6687196348.html','https://daytona.craigslist.org/cto/d/1986-porsche-911-carrera-conv/6649761594.html','https://eastoregon.craigslist.org/cto/d/early-porsche-911-project-wtb/6679741734.html','https://fargo.craigslist.org/cto/d/1983-porsche-911-garaged-with/6682299789.html','https://flint.craigslist.org/cto/d/1983-porsche-911-burgundy/6687285576.html','https://grandrapids.craigslist.org/cto/d/porsche-911-project-or-fixer/6685967106.html','https://greenville.craigslist.org/cto/d/1986-porsche-911-conv/6649701036.html','https://houston.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678163728.html','https://hudsonvalley.craigslist.org/cto/d/1982-porsche-911-slant-nose/6681725992.html','https://inlandempire.craigslist.org/cto/d/1983-porsche-911-driver-airbag/6687979723.html','https://lansing.craigslist.org/cto/d/911-porsche-looking-for-older/6687104481.html','https://lasvegas.craigslist.org/cto/d/1987-porsche-911-carrera/6687002558.html','https://lasvegas.craigslist.org/cto/d/1969-porsche-911-baja-race/6668904623.html','https://lasvegas.craigslist.org/cto/d/1977-porsche-911-sc-targa/6680151081.html','https://monterey.craigslist.org/cto/d/red-1977-porsche-911-sc-targa/6680149748.html','https://monterey.craigslist.org/ctd/d/1984-porsche-911-cab-guards/6677467366.html','https://nmi.craigslist.org/cto/d/69-porsche-911-irish-green/6656476893.html','https://norfolk.craigslist.org/cto/d/1972-porsche-911/6663694575.html','https://orlando.craigslist.org/cto/d/1991-porsche-911-carrera-2/6680295712.html','https://pensacola.craigslist.org/cto/d/1983-porsche-911-manual/6684020499.html','https://philadelphia.craigslist.org/ctd/d/1982-porsche-911-targa-1/6678187037.html','https://philadelphia.craigslist.org/cto/d/1974-porsche-911-targa/6670015872.html','https://reno.craigslist.org/cto/d/porsche-911/6683051897.html','https://sacramento.craigslist.org/cto/d/1967-porsche-911/6682711263.html']